> - 本项目利用Python的numpy和pandas进行数据清洗和处理，利用matplotlib，seaborn，pyecharts进行数据可视化展示和分析。
- 本项目数据来源Lending Club官网[2017Q2 LOAN DATA](https://www.lendingclub.com/info/download-data.action)
- 项目参考知乎专栏 [Learning Data Science - Rho](https://zhuanlan.zhihu.com/p/29651069)

## 导入项目使用的库

In [ ]:
# 处理数据
import numpy as np
import pandas as pd

# 数据可视化
# matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
# 使图片在Jupyter的cell里展现
%matplotlib inline
plt.style.use('ggplot') # 使用R语言ggplot风格的配色
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
zh_font = mpl.font_manager.FontProperties(fname = 'C:\WIndows\Fonts\msyh.ttc')

# seaborn
import seaborn as sns
sns.set_style('whitegrid') # 设置图表样式为whitegrid

# pyecharts
from pyecharts import Pie

# 忽略弹出的警告
import warnings
warnings.filterwarnings('ignore')

## 数据清洗

**载入数据**

In [ ]:
df = pd.read_csv('LoanStats_2017Q2.csv', encoding = 'latin-1', skiprows = 1)

`read_csv`参数详解可参考[这里](http://www.cnblogs.com/datablog/p/6127000.html)。

### 1. 观察数据

In [ ]:
# 数据行数和列数
# df.shape

# 查看数据信息
df.info()

In [ ]:
# 预览数据
# df.tail()
df.head()
df.delinq_2yrs.max()

通过观察数据我们知道该数据集有137个属性（列），我们本次分析主要用到的是`loan_amnt`, `term`, `int_rate`, `grade`, `issue_d`, `addr_state`, `loan_status`, `purpose`, `annual_inc`, `emp_length`。分别对应贷款金额、贷款期限、贷款利率、信用评级、业务发生时间、业务发生地、贷款状态、贷款用途、贷款人年收入和贷款人工作时间（年）。
- **取出所需数据集：**

In [ ]:
columns = ['loan_amnt', 'term', 'int_rate', 'grade', 'issue_d', 'addr_state', 
           'loan_status','purpose', 'annual_inc', 'emp_length', 'delinq_2yrs']
df_loan = df[columns]
df_loan.head()

### 2. 缺失值清洗

#### 2.1 统计每列缺失值比例

In [ ]:
df_loan.loan_amnt[df_loan.loan_amnt.isnull()]

In [ ]:
def null_count(column):     
    return len(column[column.isnull()]) / len(column)

df.apply(null_count)[df.apply(null_count) > 0.5]

因为数据集的问题好多列的缺失值的数量是巨大的，我们可以通过设置阈值来判断是否丢弃掉这些列：

In [ ]:
#  设置阈值
half_null = len(df_loan) / 2

# 删除空值数大于阈值的列(`axis = 1`)
df2 = df.dropna(thresh = half_null, axis = 1)
df2.info()

可以看出经过我们设置阈值来删除缺失值列原数据集已经由137列减少至101列，还是很多。这里只是给大家做个演示，我们并不需要这么多属性。我们需要的是上边得到的`df_loan`。我们看下所需数据集`df_loan`的缺失值情况：

In [ ]:
column_null = df_loan.apply(lambda x: x.isnull().value_counts())
column_null

可以看出数据集`df_loan`中的缺失值仅仅有两个，我们可以对所需列的缺失值进行填充，这里只需要直接删除即可（该数据集中所有缺失值都在同一行）

#### 2.2 填充缺失值

In [ ]:
df_loan = df_loan.dropna(how = 'all')
df_loan.apply(lambda x: x.isnull().value_counts())

想了解更多关于缺失值处理的知识可以参考官方文档或者[这里](http://blog.csdn.net/pipisorry/article/details/39482861)。

### 3.格式内容清洗

#### 3.1 去除数据中的空格

In [ ]:
df_loan.head()

In [ ]:
def str_trip(col):
    if col.dtype == 'O':
        return col.map(str.strip)
    else:
        return col

df_loan.apply(str_trip).head()

#### 3.2 大小写转换
把所有的州`addr_state`全部转换为大写：

In [ ]:
df_loan.addr_state = df_loan.addr_state.map(str.upper)

#### 3.3 数据格式转换
先查看数据集的数据格式：

In [ ]:
df_loan.dtypes

将贷款金额设置为整型：

In [ ]:
df_loan.loan_amnt = df_loan.loan_amnt.astype(np.int64)
df_loan.loan_amnt.dtype

后续可能还会将`issue_d`列转为时间格式，因为涉及到后边的变量分类，我们到时候再说。

### 4. 逻辑错误清洗

#### 4.1 去重
经过上边步骤的格式内容清洗后便可进行去重操作了，通过`duplicated()`判别是否有重复，DataFrame使用此方法时每行所有内容完全相同才会判重：

In [ ]:
df_loan.duplicated().value_counts()

虽然因为我们数据集取数原因（不含唯一标识），但每行中所有列的内容都相同也是基本不可能出现的，我们这里默认为其重复了（实际工作中还需要进一步判断），直接去重：

In [ ]:
df_loan = df_loan.drop_duplicates(keep = 'first')
df_loan.duplicated().value_counts()

> 更多关于pandas去重的知识可以参考[pandas标记删除重复记录](http://blog.csdn.net/kancy110/article/details/70142728)

#### 4.2 异常值清洗
通过描述统计值来判断该数据集中是否存在异常值：

In [ ]:
df_loan.describe().astype(np.int64).T

可以看到贷款金额`loan_amnt`最小值是1000美元，最大值是40000美元，处于正常范围。但年收入`annual_inc`最小值为0最大值为890万，这个有些离谱了，我们可以这些用户其他信息：

In [ ]:
# 年收入大于300万人员的信息
df_loan[df_loan.annual_inc > 3000000]

可以看到年收入大于500万的有俩人，头一个贷款16000美元已经还清了，好吧这个我们姑且不提。那个年收入890万美元的家伙贷款22000美元三年期的仅仅是为了还信用卡吗……那对不起，只能用平均收入替换您现在的收入咯。我们再来瞅瞅年收入小于1000美元的人的信息：

In [ ]:
df_loan[df_loan.annual_inc < 1000].head()

超过百分之九十的都为0，没找到Lending Club对申请人收入的要求，我们这里为了方便只好以500万为上限和1000美元为下限进行处理，处理模式为统统替换成平均值：

In [ ]:
df_loan.annual_inc = df_loan.annual_inc.replace(df_loan.annual_inc[df_loan.annual_inc>5000000],
                                               df_loan.annual_inc.mean())

df_loan.annual_inc.max()

In [ ]:
df_loan.annual_inc = df_loan.annual_inc.replace(df_loan.annual_inc[df_loan.annual_inc < 1000],
                                               df_loan.annual_inc.mean())
df_loan.annual_inc.min()

**后续如果我们作图时又发现了其他异常数据，还需要进一步处理**

## 可视化分析

### 单变量分析
我们用到的数据集有10个变量，如下：

In [ ]:
df_loan.dtypes

**1. 贷款金额（loan_amnt）分布**

In [ ]:
df_loan.loan_amnt.describe()

In [ ]:
df_loan.issue_d.head()

In [ ]:
plt.figure(figsize = (10,6))
sns.set()
sns.set_context('notebook', font_scale = 1, rc = {'lines.linewidth': 1.5})
loan_displot = sns.distplot(df_loan.loan_amnt)
plt.xlabel('贷款金额', fontproperties=zh_font)
plt.title('Lending Club贷款金额分布情况', fontproperties=zh_font)
loan_displot.figure.savefig('Loan Amount')

> 关于seaborn更多的配置可参考[Seaborn - 艺术化的图表控制](https://zhuanlan.zhihu.com/p/27435863)。本篇文章里除了涉及到`set_style`外，还有移除轴线的`despline()`，临时图表样式`axes_style()`，设置布局元素规模`plotting_context()`和`set_context()`

由上图和描述统计信息我们可知贷款金额分布呈右偏正态分布，贷款最小值为1000美元，最多为4万美元，平均1.4万左右，标准差9400，波动不是特别大。可看到Lending Club的业务主要以小额贷款为主，一般来讲大额贷款会带来高风险（当然也带来了高回报），我们得知Lending Club贷款业务带来的风险相对不会太大。

**2. 贷款周期（月）**

In [ ]:
df_loan.term.value_counts()

In [ ]:
labels = ['36 month', '60 month']
plt.pie(df_loan.term.value_counts(), labels = labels)
plt.title('贷款周期分布', fontproperties = zh_font)
plt.show()

或者我们使用pyecharts生成饼图更美观一些：

In [ ]:
pie = Pie('贷款周期分布')
pie.add('', ['36 month', '60 month'], [float(i) for i in pd.value_counts(df_loan.term)], is_label_show = True)
pie

> pyecharts更多用法可以参考[Python数据可视化](http://www.jianshu.com/p/b718c307a61c)，教程可参考pyecharts的GitHub项目[文档](https://github.com/chenjiandongx/pyecharts/blob/master/README.md)

**3. 贷款利率分布**

In [ ]:
df_loan.int_rate.dtype

我们需要把`int_rate`转换为浮点型才能进一步分析：

In [ ]:
df_loan['int_rate_num'] = df_loan.int_rate.str.rstrip('%').astype('float')
df_loan.int_rate_num.describe()

In [ ]:
plt.figure(figsize = (10,6))
sns.set()
sns.set_context('notebook', font_scale = 1, rc = {'lines.linewidth': 2})
rate_distplot = sns.distplot(df_loan.int_rate_num)
plt.xlabel('贷款利率', fontproperties = zh_font)
plt.title('贷款利率分布图', fontproperties = zh_font)
rate_distplot.figure.savefig('Interest Rate')

由描述统计信息和利率分布图可知贷款利率呈右偏正态分布，利率中位数12.62%，最低值5.32%，最高值30.99%...，平均13.29%。我们贷款利率越高借款人还款成本越高，违约的可能性就越高。

**4. 客户信用等级分布**

In [ ]:
df_loan.grade.value_counts()

In [ ]:
attr = ['C', 'B', 'A', 'D', 'E', 'F', 'G']
pie = Pie('客户信用等级分布情况')
pie.add('', attr, [float(i) for i in df_loan.grade.value_counts()], is_label_show = True)
pie

可以看出ABC三类等级的客户共占了81.62%，Lending Club对申请人的资信情况把关还是比较严格的。

**5. 业务地区分布**

In [ ]:
df_loan.addr_state.value_counts().head()

In [ ]:
plt.figure(figsize = (16, 6))
sns.set()
sns.set_context('notebook', font_scale = 1.5, rc = {'lines.linewidth': 2})
ax = sns.countplot(x = 'addr_state', data = df_loan, palette = 'Set2')
#ax.set(yscale = 'log')
plt.xticks(rotation = 45)
plt.title('贷款业务地区分布', fontproperties = zh_font)
plt.show()

**6. 贷款状态**

In [ ]:
df.loan_status.value_counts()

贷款状态往大了说其实就两种，正常和违约，但可以看到上边出现了7种情况，需要我们针对这个分类变量进行编码：

In [ ]:
def categorical_coded(col, dict):
    col = pd.Series(col, copy = True)
    
    for key, value in dict.items():
        col.replace(key, value, inplace = True)
    
    return col

df_loan['loan_status_cate'] = categorical_coded(df.loan_status, 
                                                {
                                                    'Current': 0,
                                                    'Fully Paid': 0,
                                                    'In Grace Period': 1,
                                                    'Late (31-120 days)': 1,
                                                    'Late (16-30 days)': 1,
                                                    'Charged Off': 1,
                                                    'Default': 0
                                                })
df_loan.loan_status_cate.value_counts()

In [ ]:
attr = ['正常', '违约']
pie = Pie('贷款状态占比')
pie.add('', attr, [int(i) for i in df_loan.loan_status_cate.value_counts()], is_label_show = True)
pie

**7. 贷款用途**

In [ ]:
df_loan.purpose.value_counts()

In [ ]:
plt.figure(figsize = (10, 6))
sns.set()
sns.set_context('notebook', font_scale = 1.5, rc = {'lines.linewidth': 2})
ax = sns.countplot(x = 'purpose', data = df_loan, palette = 'Set3')
ax.set(yscale = 'log')
plt.xticks(rotation = 45)
plt.title('贷款用途', fontproperties = zh_font)
plt.show()

贷款用途前三：债务重组（借钱还旧债，好像说的是我...），信用卡还款，住房改善。通常来讲债务重组和信用卡还款的客户现金流较紧张，还款能力相对较弱，发生违约的可能性较高，公司可以做好风控的同时也能拓展更多此类客户。

### 多变量分析

**1. 贷款金额和时间**

In [ ]:
df_loan['issue_date'] = pd.to_datetime(df_loan.issue_d)
df_loan.head()

In [ ]:
loan_amnt_groupby_date = df_loan.groupby('issue_date').sum().reset_index()
loan_amnt_groupby_date['issue_m'] = loan_amnt_groupby_date.issue_date.apply(lambda x: x.to_period('M'))
loan_amnt_groupby_date

In [ ]:
plt.figure(figsize = (6,4))
sns.set()
sns.set_context('notebook', font_scale = 1, rc={'lines.linewidth': 2})
month_barplot = sns.barplot(x='issue_m', y='loan_amnt', data = loan_amnt_groupby_date)
plt.xlabel('月份', fontproperties = zh_font)
plt.ylabel('贷款总额', fontproperties = zh_font)
plt.title('不同月份贷款总额的分布', fontproperties = zh_font)

4月份业务总额最低，但因为我们只拿到了Q2季度的数据，没办法进行同比、环比等操作来进行进一步的分析，对实际指导没有太大意义。

**2. 贷款金额和地区**

In [ ]:
loan_amnt_groupby_state = df_loan.groupby('addr_state').sum().reset_index()
loan_amnt_groupby_state.head()

In [ ]:
plt.figure(figsize = (16, 6))
sns.set()
sns.set_context('notebook', rc = {'lines.linewidth': 2})
state_barplot = sns.barplot(x = 'addr_state', y = 'loan_amnt', data = loan_amnt_groupby_state)
plt.xlabel('州', fontproperties = zh_font)
plt.ylabel('贷款总额', fontproperties = zh_font)
plt.title('不同地区的贷款总额', fontproperties = zh_font)

加州、德州和纽约州的业务要远高于其他州，同时也要加大这几个地区贷款申请人的信息审核力度。如果有不同月份数据的话我们还可以观察下这些地区不同月份的业绩走向。

**3. 信用等级、贷款期限和利率**

In [ ]:
rate_pivot = df_loan.pivot_table(index = 'term', columns = 'grade', values = 'int_rate_num', aggfunc = 'mean')
rate_pivot

In [ ]:
plt.figure(figsize=(10, 6))
sns.set_context('notebook', rc={'lines.linewidth': 2})
rate_boxplot = sns.boxplot(y='grade', x='int_rate_num', data=df_loan)
sns.despine(top=True)
plt.xlabel('贷款利率', fontproperties = zh_font)
plt.ylabel('信用评级', fontproperties = zh_font)
plt.title('信用等级和利率的关系', fontproperties = zh_font)

从图中可以看出信用等级越差的用户贷款利率也会越高，这也是为了控制风险的一种方式。

**4. 贷款用途和利率**

In [ ]:
plt.figure(figsize=(10, 6))
sns.set_context('notebook', font_scale = 1.5, rc={'lines.linewidth': 2.5})
pubpose_boxplot = sns.boxplot(y='purpose', x='int_rate_num', data=df_loan)
plt.xlabel('贷款利率', fontproperties = zh_font)
plt.ylabel('贷款用途图', fontproperties = zh_font)
plt.title('贷款用途和利率的关系', fontproperties = zh_font)

**5. 贷款金额和利率**

In [ ]:
plt.figure(figsize=(10, 6))
amnt_rate_join = sns.jointplot('loan_amnt', 'int_rate_num', data=df_loan, kind='reg',size=6)

图中可以看出皮尔森相关系数为0.14，两者没有太大的相关性

**6. 违约次数和贷款利率**

In [ ]:
df_loan.delinq_2yrs = df_loan.delinq_2yrs.astype('int')
df_loan.tail()

In [ ]:
plt.figure(figsize=(10, 6))
sns.set_context('notebook', rc={'lines.linewidth': 2.5})
sboxplot = sns.boxplot(x="delinq_2yrs", y="int_rate_num", data=df_loan)
plt.xlabel('违约次数', fontproperties = zh_font)
plt.ylabel('贷款利率', fontproperties = zh_font)
plt.title('违约次数和贷款利率', fontproperties = zh_font)

箱线图好像不是很明显，我们可以看下这两者的线性关系：

In [ ]:
df_loan.int_rate_num = df_loan.int_rate_num / 100
sns.lmplot(x='delinq_2yrs', y='int_rate_num', data=df_loan)
plt.xlabel('违约次数', fontproperties = zh_font)
plt.ylabel('贷款利率', fontproperties = zh_font)
plt.title('违约次数和贷款利率的线性关系', fontproperties = zh_font)

可以看出贷款利率大体上还是随着违约次数的增加而增加的。

**7. 利率、收入、工作年限以及贷款状态**

In [ ]:
df_loan.emp_length.value_counts()

为了方便分析，我们需要把工作时间转为数值型数据：

In [ ]:
dict = {
    'emp_length': {
        '10+ years': 10,
        '9 years': 9,
        '8 years': 8,
        '7 years': 7,
        '6 years': 6,
        '5 years': 5,
        '4 years': 4,
        '3 years': 3,
        '2 years': 2,
        '1 year': 1,
        '< 1 year': 0,
        'n/a': 0
    }
}

df_loan = df_loan.replace(dict)
df_loan.emp_length.value_counts()

In [ ]:
sns.set_context('notebook', font_scale=2, rc={'lines.linewidth': 2.5})
sns.pairplot(df_loan, vars=['int_rate_num','annual_inc', 'emp_length'], 
             hue='loan_status_cate', diag_kind='kde', kind='reg', size = 7)

**8. 绘制相关系数图**

In [ ]:
df_loan.corr()

In [ ]:
names = ['loan_amnt', 'annual_inc', 'emp_length', 'int_rate_num', 'loan_status_cate']
correlations = df_loan.corr()

fig = plt.figure(figsize = (10, 9))
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin = -1, vmax = 1)
fig.colorbar(cax)
ticks = np.arange(0, 5, 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(names)
ax.set_yticklabels(names)
plt.xticks(rotation = 45)
plt.show()